In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [0]:
datos=pd.read_csv('https://raw.githubusercontent.com/luis4god/selenium_extraccion_de_datos_modelaje_y_entrenamientos_keras-tensorflow-backend-/Dev_ops/ejemplos_de_clarin.csv')

In [5]:
datos.head()

,Unnamed: 0,subtitulo,ejemplos
0,0,Departamentos(3014),Departamentos 3014
1,1,Departamentos(3014),7876468 CAMARONES 2761 Villa Sta Rita
2,2,Departamentos(3014),7873862 CHACARITA 2amb u s 140 000 A metro...
3,3,Departamentos(3014),7873865 CABALLITO 3amb COCH FIJA SUM pis...
4,4,Departamentos(3014),7874011 PALERMO 2amb Billinghurst y Avda C...


In [0]:
datos = datos[['subtitulo','ejemplos']]

In [7]:
datos.head()

,subtitulo,ejemplos
0,Departamentos(3014),Departamentos 3014
1,Departamentos(3014),7876468 CAMARONES 2761 Villa Sta Rita
2,Departamentos(3014),7873862 CHACARITA 2amb u s 140 000 A metro...
3,Departamentos(3014),7873865 CABALLITO 3amb COCH FIJA SUM pis...
4,Departamentos(3014),7874011 PALERMO 2amb Billinghurst y Avda C...


In [0]:
datos['ejemplos'] = datos['ejemplos'].apply(lambda x: x.lower())
datos['ejemplos'] = datos['ejemplos'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [9]:
datos['ejemplos'].shape

(2451,)

In [0]:
maxim = datos['ejemplos'].shape[0]

tokenizer = Tokenizer(num_words=maxim, split=' ')
tokenizer.fit_on_texts(datos['ejemplos'].values)
X = tokenizer.texts_to_sequences(datos['ejemplos'].values)
X = pad_sequences(X)

In [0]:
y=[x for x in datos['subtitulo']]

labelencoder_y=LabelEncoder()
labelencoder_y.fit(np.asarray(y))
yi=labelencoder_y.transform(np.asarray(y))

In [12]:
clases={}
for i in range(len(yi)):
    clases[yi[i]]=y[i]
print(len(yi))

2451


In [13]:
clases

{0: '4x4, Pick-up, Vans, Mini Vans, Utilitarios(116)',
 1: 'Astrología y Tarot(152)',
 2: 'Automóviles Nacionales e Importados(699)',
 3: 'Boliches y Otros(12)',
 4: 'Casas(388)',
 5: 'Choferes, Personal de Transporte y Abastecimiento, Autos(302)',
 6: 'Construcción y Refacciones(58)',
 7: 'Cuidado de Personas(463)',
 8: 'Departamentos(3014)',
 9: 'Empleados(88)',
 10: 'Habitaciones en Hoteles, Casas de Familias y Pensiones(190)',
 11: 'Hogar y Muebles(14)',
 12: 'Locales Comerciales. Oficinas y Consultorios(307)',
 13: 'Mascotas y Animales(20)',
 14: 'Mudanzas, Fletes y Logística(67)',
 15: 'Máquinas y otros materiales para industrias y negocios(32)',
 16: 'Oficios y Ocupaciones Varias(679)',
 17: 'Personal auxiliar de Casas Particulares. Hoteles, Clínicas(91)',
 18: 'Planes de ahorro y Chocados(31)',
 19: 'Profesionales y Ejecutivos(87)',
 20: 'Préstamos, Hípotecas y Otros(178)',
 21: 'Servicios Automotores y Choferes(36)',
 22: 'Sex shops(13)',
 23: 'Taxis, Repuestos y Accesorios(34

In [14]:
cx=yi.reshape(len(X),1).astype("float32")
onehotencoder=OneHotEncoder(categorical_features=[0])
Y=onehotencoder.fit_transform(cx).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [15]:
yi.shape

(2451,)

In [0]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 55)

In [19]:
model = Sequential()
model.add(Embedding(maxim, 20,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(25,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.fit(X_train, Y_train, epochs = 3, batch_size=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
1642/1642 [==============================] - 11s 7ms/step - loss: 3.1017 - acc: 0.0895
Epoch 2/3
1642/1642 [==============================] - 10s 6ms/step - loss: 2.9465 - acc: 0.0938
Epoch 3/3
1642/1642 [==============================] - 10s 6ms/step - loss: 2.9329 - acc: 0.1005


In [0]:
#model.save_weights('pesos_nlp_lstm_embeddings.h5')
#model.load_weights('pesos_nlp_lstm_embeddings.h5')

In [22]:
score,acc = model.evaluate(X_test, Y_test, batch_size =100)

809/809 [==============================] - 1s 2ms/step


In [23]:
print("score={},acc={}".format(score,acc))

score=2.884068849797007,acc=0.08776266988003062


In [0]:
word=["notebook"]
word=tokenizer.texts_to_sequences(word)
word=pad_sequences(word, maxlen=106, dtype='int32', value=0)

In [28]:
clases[int(model.predict_classes(word))]

'Departamentos(3014)'